In [1]:
import copy
from collections import deque
import seaborn as sns
import matplotlib.pyplot as plt

class PuzzleState:
    def __init__(self, grid, depth=0, parent=None):
        self.grid = grid
        self.depth = depth
        self.parent = parent

    def __eq__(self, other):
        return self.grid == other.grid

    def find_blank(self):
        for i in range(3):
            for j in range(3):
                if self.grid[i][j] == 0:
                    return i, j

    def generate_successors(self):
        successors = []
        x, y = self.find_blank()
        moves = [(-1,0), (1,0), (0,-1), (0,1)]  # up, down, left, right
        for dx, dy in moves:
            nx, ny = x + dx, y + dy
            if 0 <= nx < 3 and 0 <= ny < 3:
                new_grid = copy.deepcopy(self.grid)
                new_grid[x][y], new_grid[nx][ny] = new_grid[nx][ny], new_grid[x][y]
                successors.append(PuzzleState(new_grid, self.depth+1, self))
        return successors


# -------- Recursive DFS --------
def solve_dfs(state, goal_state, visited, path, max_depth=50):
    if any(state == v for v in visited):
        return None

    visited.append(state)
    path.append(state)

    if state.grid == goal_state:
        return path.copy()

    if state.depth >= max_depth:
        path.pop()
        return None

    for neighbor in state.generate_successors():
        result = solve_dfs(neighbor, goal_state, visited, path, max_depth)
        if result:
            return result

    path.pop()
    return None


# -------- Recursive BFS --------
def solve_bfs(queue, goal_state, visited):
    if not queue:
        return None

    state, path = queue.popleft()

    if state.grid == goal_state:
        return path

    for neighbor in state.generate_successors():
        if not any(neighbor == v for v in visited):
            visited.append(neighbor)
            queue.append((neighbor, path + [neighbor]))

    return solve_bfs(queue, goal_state, visited)


# -------- Output Functions --------
def print_path_text(path, algo_name):
    print(f"\n{algo_name} Solution Path:")
    for state in path:
        print(f"Depth: {state.depth}")
        for row in state.grid:
            print(" ".join(map(str, row)))
        print("--------")

def plot_path_seaborn(path, algo_name):
    for state in path:
        plt.figure(figsize=(3,3))
        sns.heatmap(state.grid, annot=True, cbar=False, square=True,
                    linewidths=1, linecolor="black", cmap="Blues",
                    fmt="d", annot_kws={"size":16})
        plt.title(f"{algo_name} - Depth {state.depth}")
        plt.show()

In [2]:
initial_state = [[2, 8, 3],
                 [1, 6, 4],
                 [7, 0, 5]]

final_goal_state = [[1, 2, 3],
                    [8, 0, 4],
                    [7, 6, 5]]

start = PuzzleState(initial_state)

In [3]:
visited = []
path = []
dfs_solution = solve_dfs(start, final_goal_state, visited, path, max_depth=30)

visited = [start]
queue = deque([(start, [start])])
bfs_solution = solve_bfs(queue, final_goal_state, visited)

In [4]:
print_path_text(dfs_solution, "Recursive DFS")


Recursive DFS Solution Path:
Depth: 0
2 8 3
1 6 4
7 0 5
--------
Depth: 1
2 8 3
1 0 4
7 6 5
--------
Depth: 2
2 0 3
1 8 4
7 6 5
--------
Depth: 3
0 2 3
1 8 4
7 6 5
--------
Depth: 4
1 2 3
0 8 4
7 6 5
--------
Depth: 5
1 2 3
8 0 4
7 6 5
--------


In [5]:
print_path_text(bfs_solution, "Recursive BFS")


Recursive BFS Solution Path:
Depth: 0
2 8 3
1 6 4
7 0 5
--------
Depth: 1
2 8 3
1 0 4
7 6 5
--------
Depth: 2
2 0 3
1 8 4
7 6 5
--------
Depth: 3
0 2 3
1 8 4
7 6 5
--------
Depth: 4
1 2 3
0 8 4
7 6 5
--------
Depth: 5
1 2 3
8 0 4
7 6 5
--------
